In [7]:
gcloud auth login

SyntaxError: invalid syntax (2679669346.py, line 1)

In [2]:
!uv pip install google-cloud-bigquery

Audited 1 package in 7ms


In [1]:
import os
os.environ["GOOGLE_CLOUD_PROJECT"] = "dev-tradyon-data"
print("Using project:", os.environ["GOOGLE_CLOUD_PROJECT"])

Using project: dev-tradyon-data


In [10]:
from google.cloud import bigquery

client = bigquery.Client()

#  Fetch one row
query = "SELECT * FROM `dev-tradyon-data.tradyon.product_master` LIMIT 1"
rows = client.query_and_wait(query)
for row in rows:
    sample_row = dict(row)
    print("Sample row:", sample_row)

# 2️ Fetch schema directly from table object
table = client.get_table("dev-tradyon-data.tradyon.product_master")
print("\nSchema:")
for field in table.schema:
    print(f"{field.name} ({field.field_type})")


Sample row: {'product_id': 'p3', 'product_name': 'Cubeb Pepper', 'synonymns': 'Piper Cubeba, Litsea Cubeba, Wild Forest Pepper', 'product_schema': {'Organic': ['Organic'], 'Origin_Country': ['Indonesia', 'Vietnam'], 'Packaging_Type': ['Bag', 'Box', 'Barrel'], 'Physical_Form': ['Whole', 'Berries'], 'Processing_Method': ['Roasted', 'Eto Sterilized', 'Dried'], 'Variety': ['Cubeb Pepper', 'Litsea Cubeba', 'Piper Cubeba', 'Wild Forest Pepper']}}

Schema:
product_id (STRING)
product_name (STRING)
synonymns (STRING)
product_schema (JSON)

Schema:
product_id (STRING)
product_name (STRING)
synonymns (STRING)
product_schema (JSON)


In [11]:
from google.cloud import bigquery

client = bigquery.Client()

# 1️ Fetch one row
query = "SELECT * FROM `dev-tradyon-data.tradyon.shipment_master_taxonomy` LIMIT 1"
rows = client.query_and_wait(query)
for row in rows:
    sample_row = dict(row)
    print("Sample row:", sample_row)

# 2️ Fetch schema directly from table object
table = client.get_table("dev-tradyon-data.tradyon.shipment_master_taxonomy")
print("\nSchema:")
for field in table.schema:
    print(f"{field.name} ({field.field_type})")


Sample row: {'date': datetime.datetime(2024, 7, 31, 0, 0), 'hs_code': '09041120', 'goods_shipped': None, 'shipment_destination': 'Vietnam', 'shipment_origin': 'Vietnam', 'port_of_lading': None, 'port_of_lading_country': None, 'port_of_lading_un_locode': None, 'port_of_unlading': None, 'port_of_unlading_country': None, 'port_of_unlading_un_locode': None, 'trade_direction': 'Export', 'transport_method': 'Air', 'is_containerized': 'No', 'value_of_goods_usd': 16.0, 'industry_gics': '30202030', 'shipment_id': '580003309a97bd68bb874b5089a351050ace10304dfbb8f4ff38ad2f5f409d1a', 'is_multi_product_shipment': False, 'volume_teu': None, 'weight_in_kg': None, 'shipper_profile': None, 'shipper_ultimate_parent_profile': None, 'consignee_profile': None, 'consignee_ultimate_parent_profile': None, 'consignee_tradyon_profile_id': None, 'shipper_tradyon_profile_id': None, 'consignee_business_id': None, 'shipper_business_id': None, 'unit_price': None, 'original_consignee_business_id': None, 'original_ship

In [2]:

from google.cloud import bigquery
import csv

client = bigquery.Client()

csv_path = '/home/parshav-potato/Work/tradyon/generic_pipeline/sample_onion.csv'

data = []
with open(csv_path, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        cleaned = {k: (v if v != '' else None) for k, v in row.items()}
        if cleaned.get('is_multi_product_shipment') is not None:
            cleaned['is_multi_product_shipment'] = cleaned['is_multi_product_shipment'].lower() == 'true'
        # Drop attributes_struct if present so schema matches table definition
        cleaned.pop('attributes_struct', None)
        data.append(cleaned)

if not data:
    raise ValueError('No rows found in sample_onion.csv')

schema = [
    bigquery.SchemaField('date', 'DATETIME'),
    bigquery.SchemaField('hs_code', 'STRING'),
    bigquery.SchemaField('goods_shipped', 'STRING'),
    bigquery.SchemaField('shipment_destination', 'STRING'),
    bigquery.SchemaField('shipment_origin', 'STRING'),
    bigquery.SchemaField('port_of_lading', 'STRING'),
    bigquery.SchemaField('port_of_lading_country', 'STRING'),
    bigquery.SchemaField('port_of_lading_un_locode', 'STRING'),
    bigquery.SchemaField('port_of_unlading', 'STRING'),
    bigquery.SchemaField('port_of_unlading_country', 'STRING'),
    bigquery.SchemaField('port_of_unlading_un_locode', 'STRING'),
    bigquery.SchemaField('trade_direction', 'STRING'),
    bigquery.SchemaField('transport_method', 'STRING'),
    bigquery.SchemaField('is_containerized', 'STRING'),
    bigquery.SchemaField('value_of_goods_usd', 'FLOAT'),
    bigquery.SchemaField('industry_gics', 'STRING'),
    bigquery.SchemaField('shipment_id', 'STRING'),
    bigquery.SchemaField('is_multi_product_shipment', 'BOOLEAN'),
    bigquery.SchemaField('volume_teu', 'FLOAT'),
    bigquery.SchemaField('weight_in_kg', 'FLOAT'),
    bigquery.SchemaField('shipper_profile', 'STRING'),
    bigquery.SchemaField('shipper_ultimate_parent_profile', 'STRING'),
    bigquery.SchemaField('consignee_profile', 'STRING'),
    bigquery.SchemaField('consignee_ultimate_parent_profile', 'STRING'),
    bigquery.SchemaField('consignee_tradyon_profile_id', 'INTEGER'),
    bigquery.SchemaField('shipper_tradyon_profile_id', 'INTEGER'),
    bigquery.SchemaField('consignee_business_id', 'STRING'),
    bigquery.SchemaField('shipper_business_id', 'STRING'),
    bigquery.SchemaField('unit_price', 'FLOAT'),
    bigquery.SchemaField('original_consignee_business_id', 'STRING'),
    bigquery.SchemaField('original_shipper_business_id', 'STRING'),
    bigquery.SchemaField('product_id', 'STRING'),
    bigquery.SchemaField('product_name', 'STRING'),
    bigquery.SchemaField('attributes', 'JSON'),
    bigquery.SchemaField('is_outlier', 'BOOLEAN'),
    bigquery.SchemaField('outlier_type', 'STRING'),
]

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
    schema=schema,
)

table_id = 'dev-tradyon-data.tradyon.shipment_master_taxonomy'

job = client.load_table_from_json(data, table_id, job_config=job_config)
job.result()

print(f'Appended {len(data)} rows to {table_id}')


Appended 25157 rows to dev-tradyon-data.tradyon.shipment_master_taxonomy


In [3]:
from google.cloud import bigquery
import csv
import json

client = bigquery.Client()

csv_path = '/home/parshav-potato/Work/tradyon/generic_pipeline/output/onion/per_product_classifications/output/onion/shipment_id_to_attr.csv'

# Read CSV and aggregate by shipment_id
csv_updates = {}
with open(csv_path, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        shipment_id = (row.get('shipment_id') or '').strip()
        if not shipment_id:
            continue
        
        raw_product = (row.get('product') or '').strip()
        product_name = raw_product if raw_product and raw_product.lower() != 'none' else None
        
        attrs_raw = (row.get('attrs_json') or '').strip()
        attrs = json.loads(attrs_raw) if attrs_raw else None
        
        # Initialize or get existing entry
        existing = csv_updates.setdefault(shipment_id, {
            'shipment_id': shipment_id,
            'product_name': None,
            'attributes': {}
        })
        
        # Update product_name if not already set
        if product_name and not existing['product_name']:
            existing['product_name'] = product_name
        
        # Merge attributes
        if attrs:
            existing['attributes'].update(attrs)

# Filter to only rows with actual data
csv_updates = {k: v for k, v in csv_updates.items() if v['product_name'] or v['attributes']}

if not csv_updates:
    raise ValueError('No applicable shipment rows found to update.')

print(f'Found {len(csv_updates)} shipment IDs to update from CSV')

# Fetch existing data from BigQuery for these shipment IDs
shipment_ids = list(csv_updates.keys())
batch_size = 1000
existing_data = {}

for i in range(0, len(shipment_ids), batch_size):
    batch = shipment_ids[i:i + batch_size]
    ids_str = ', '.join([f"'{sid}'" for sid in batch])
    
    query = f"""
    SELECT shipment_id, product_name, attributes
    FROM `dev-tradyon-data.tradyon.shipment_master_taxonomy`
    WHERE shipment_id IN ({ids_str})
    """
    
    results = client.query(query).result()
    for row in results:
        existing_data[row.shipment_id] = {
            'product_name': row.product_name,
            'attributes': dict(row.attributes) if row.attributes else {}
        }

print(f'Fetched {len(existing_data)} existing records from BigQuery')

# Merge CSV data with existing data
final_updates = []
for shipment_id, csv_data in csv_updates.items():
    existing = existing_data.get(shipment_id, {'product_name': None, 'attributes': {}})
    
    # Merge attributes (CSV data takes precedence)
    merged_attributes = dict(existing['attributes'])
    merged_attributes.update(csv_data['attributes'])
    
    final_updates.append({
        'shipment_id': shipment_id,
        'product_name': csv_data['product_name'] or existing['product_name'],
        'attributes': merged_attributes if merged_attributes else None
    })

# Create temporary table with merged updates
temp_table_id = 'dev-tradyon-data.tradyon._shipment_attr_updates'

load_job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    schema=[
        bigquery.SchemaField('shipment_id', 'STRING'),
        bigquery.SchemaField('product_name', 'STRING'),
        bigquery.SchemaField('attributes', 'JSON'),
    ],
)

load_job = client.load_table_from_json(final_updates, temp_table_id, job_config=load_job_config)
load_job.result()

# Merge updates into main table
merge_query = """
MERGE `dev-tradyon-data.tradyon.shipment_master_taxonomy` T
USING `dev-tradyon-data.tradyon._shipment_attr_updates` S
ON T.shipment_id = S.shipment_id
WHEN MATCHED THEN
  UPDATE SET
    product_name = S.product_name,
    attributes = S.attributes
"""

merge_result = client.query(merge_query).result()

# Clean up temporary table
client.delete_table(temp_table_id, not_found_ok=True)

print(f'Successfully updated {len(final_updates)} shipment records in BigQuery')


Found 17778 shipment IDs to update from CSV
Fetched 17778 existing records from BigQuery
Successfully updated 17778 shipment records in BigQuery


In [4]:
from google.cloud import bigquery

client = bigquery.Client()

# Delete temporary table
temp_table_id = 'dev-tradyon-data.tradyon._shipment_attr_updates'

try:
    client.delete_table(temp_table_id, not_found_ok=True)
    print(f'Deleted temporary table: {temp_table_id}')
except Exception as e:
    print(f'Error deleting table: {e}')


Deleted temporary table: dev-tradyon-data.tradyon._shipment_attr_updates


## Load product schema master
Appends rows from `output/product_schema_master.csv` into `tradyon.product_master` with normalized schema keys.

In [5]:
from google.cloud import bigquery
import csv
import json

client = bigquery.Client()

csv_path = '/home/parshav-potato/Work/tradyon/generic_pipeline/output/onion/product_schema_master.csv'

records = []
with open(csv_path, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        raw_schema = json.loads(row['schema_json'])
        # Normalize keys to Title_Case with underscores for consistency
        normalized_schema = {
            '_'.join(part.capitalize() for part in key.split('_')): value
            for key, value in raw_schema.items()
        }
        records.append({
            'product_id': row['product_id'],
            'product_name': row['product'],
            'synonymns': '',
            'product_schema': normalized_schema
        })

if not records:
    raise ValueError('No rows found in product_schema_master.csv to append.')

table_id = 'dev-tradyon-data.tradyon.product_master'

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
    schema=[
        bigquery.SchemaField('product_id', 'STRING'),
        bigquery.SchemaField('product_name', 'STRING'),
        bigquery.SchemaField('synonymns', 'STRING'),
        bigquery.SchemaField('product_schema', 'JSON'),
    ],
)

job = client.load_table_from_json(records, table_id, job_config=job_config)
job.result()

print(f'Appended {len(records)} rows to {table_id}')


Appended 1 rows to dev-tradyon-data.tradyon.product_master
